In [1]:
import polars as pl
from datasets.token_price import TokenDataset
from models.simple_nn import SimpleNN
from models.linear_regression import LinearRegression
from models.utils import convert_to_onnx
from training.evaluation import k_fold_cross_validation
import torch.optim as optim
import torch.nn as nn
import torch
import time
import proof.ezkl as ezkl
from utils import green

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from skl2onnx import to_onnx

In [2]:
TARGET_LAG = 1 
TOKEN_NAME = "WETH" # Choose one of the available tokens in the main dataset.
STARTER_DATE = pl.datetime(2022, 6, 1)

In [3]:
# Split the dataset into training and testing sets.
x_train, x_test, y_train, y_test = TokenDataset.get_train_test_split(TOKEN_NAME, STARTER_DATE, TARGET_LAG, split_ratio=0.75)

Dataset read from cache.
Loading dataset tokens-daily-prices-mcap-volume from cache.
Dataset read from cache.
Loading dataset top-pools-apy-per-protocol from cache.
Dataset read from cache.
Loading dataset tvl-per-project-tokens from cache.


In [4]:
# Get some information about the dataset.
print("First 5 rows of the training set:")
print(x_train.head(5))

x_test[int(len(x_test) * 0.6):].write_csv("./example_token_trend.csv")

First 5 rows of the training set:
shape: (5, 120)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ tvlUsd_pr ┆ apy_proje ┆ apy_proje ┆ apy_proje ┆ … ┆ pendle   ┆ diff_pric ┆ rocket-po ┆ tvlUsd_pr │
│ oject_sus ┆ ct_uniswa ┆ ct_sushis ┆ ct_uniswa ┆   ┆ ---      ┆ e_1_days_ ┆ ol        ┆ oject_sus │
│ hiswap_Et ┆ p-v3_Arbi ┆ wap_Ether ┆ p-v2_Ethe ┆   ┆ f64      ┆ agoWETH   ┆ ---       ┆ hiswap_Et │
│ hereu…    ┆ trumW…    ┆ eumWE…    ┆ reumX…    ┆   ┆          ┆ ---       ┆ f64       ┆ hereu…    │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆          ┆ f64       ┆           ┆ ---       │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆          ┆           ┆           ┆ f64       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 1.787736  ┆ -0.520166 ┆ -0.125548 ┆ -0.319669 ┆ … ┆ 0.162243 ┆ -2.144109 ┆ -0.853152 ┆ 4.647859  │
│ 1.607506  ┆ -0.549403 ┆ -0.186733 ┆ 0.0

In [5]:
# Convert the data to PyTorch tensors.
x_train, x_test, y_train, y_test = tuple(
    map(lambda x: torch.tensor(x.to_numpy(), dtype=torch.float32), [x_train, x_test, y_train, y_test])
)

In [6]:
input_size = x_train.shape[1]

PYTORCH_MODELS = {
    "Neural Network": k_fold_cross_validation(x_train, y_train, SimpleNN(input_size), nn.BCELoss(), optim.Adam),
    "Linear Regression": k_fold_cross_validation(x_train, y_train, LinearRegression(input_size), nn.MSELoss(), optim.Adam)
}

y_train_as_1d_array = y_train.squeeze().numpy()
y_test_as_1d_array = y_test.squeeze().numpy()

SKLEARN_MODELS = {
    "Decision Tree": DecisionTreeClassifier(criterion="log_loss", max_depth=5),
    "Logistic Regression": LogisticRegression(max_iter=100),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC()
}

SKLEARN_MODELS = {
    model_name: model.fit(x_train, y_train_as_1d_array) 
    for model_name, model in SKLEARN_MODELS.items()
}

SKLEARN_MODELS = {
    model_name: (model, model.score(x_test, y_test_as_1d_array)) 
    for model_name, model in SKLEARN_MODELS.items()
}

In [7]:
# Print the results of the models.
for model_name, (model, score) in (SKLEARN_MODELS | PYTORCH_MODELS).items():
    print(f"{model_name} - Accuracy: {green(str(score)[:4])}")

Decision Tree - Accuracy: 0.52
Logistic Regression - Accuracy: 0.58
Random Forest - Accuracy: 0.53
SVM - Accuracy: 0.59
Neural Network - Accuracy: 0.66
Linear Regression - Accuracy: 0.74


In [8]:
# Save as ONNX
ONNX_PATH = "onnx/"
path = lambda model_name: ONNX_PATH + model_name.replace(" ", "_").lower() + ".onnx"
shape = x_test.shape[1:]
sample = torch.rand(1, *shape, requires_grad=True)

for model_name, (model, _) in PYTORCH_MODELS.items():
    convert_to_onnx(model, sample, path(model_name))

for model_name, (model, _) in SKLEARN_MODELS.items():
    onx = to_onnx(model, sample.detach().numpy())
    with open(path(model_name), "wb") as f:
        f.write(onx.SerializeToString())

## EZKL Proof  

In [9]:
bench = lambda model, model_name: ezkl.bench_ezkl(model, path(model_name), sample, shape, 1)

for model_name, (model, _) in (PYTORCH_MODELS | SKLEARN_MODELS).items():
    print(f"{model_name} proof:")
    bench(model, model_name)
    print()

Decision Tree proof:


RuntimeError: Failed to generate settings: Translating node #3 "ZipMap" Unimplemented(ZipMap) ToTypedTranslator

In [ ]:
ezkl.setup(
    model1, 
    MODEL_ONNX,
    sample, 
    shape
)

In [ ]:
start = time.time()
ezkl.prove()
end = time.time()
print(f"Proving time: {green(str(end - start)[:5] + ' [s]')}")

In [ ]:
start = time.time()
ezkl.verify()
end = time.time()
print(f"Verification time: {green(str(end - start)[:5] + ' [s]')}")

## Giza Proof 

In [ ]:
# Transpile the models to cairo using Giza
! giza transpile onnx/model1.onnx --output-path model

In [ ]:
# Deploy the endpoint to Giza
! giza endpoints deploy --model-id 584 --version-id 3

In [ ]:
! giza workspaces get

# 🚨 If you haven't set up a workspace yet, you can establish one by executing the command below:
# `! giza workspaces create`

In [ ]:
from giza_actions.action import action
from giza_actions.model import GizaModel
from giza_actions.task import task
import polars as pl
import numpy as np


@task(name="Prediction with Cairo")
def prediction(X_test, model):
    (result, request_id) = model.predict(
        input_feed={"input": X_test},
        verifiable=True,
        custom_output_dtype="arr_fixed_point",
    )
    return result, request_id


@action(name="Execution: Prediction with Cairo", log_prints=True)
def execution():
    model = GizaModel(id=584, version=3)
    df = pl.read_csv("./example_token_trend.csv")
    model_input = df.to_numpy().astype(np.float32)
    (result, request_id) = prediction(model_input, model)
    return result, request_id


(result, request_id) = execution()
print(f"Result: {result}, Request ID: {request_id}")